In [ ]:
# Started w/ circle_routing_11.ipynb -- consolidation & fixing it up
import pandas as pd
import polars as pl
import numpy  as np
import networkx as nx
import random
from rtsvg import *
rt = RACETrack()
my_df = pl.DataFrame({
    'fm':'a b c d e f g h i j k l m n o a b c d e f g h i j k l m n o h h'.split(),
    'to':'b c d e f g h i j k l m n o a c d d c b b k l k l l m n m m j k'.split()})
my_pos = {'m': (4.756287622381761, -4.648929693706887),
 'n': (5.8766434111262935, 3.9477905470719783),
 'o': (2.8740898972909434, 3.8082334003060865),
 'a': (1.6626394716232904, 2.0848839532480787),
 'c': (-0.5933891572168369, 2.5998077658181966),
 'd': (-4.654701003072322, 3.9198791177188),
 'b': (-0.29669457860841847, 1.2999038829090983),
 'h': (-6.006459119349462, -2.892558260783717),
 'l': (-1.7193688365616442, -4.7605754111196),
 'i': (-3.8928590667260394, -2.946332503163021),
 'j': (-2.660467699107052, -3.4766496608734054),
 'k': (-1.764183068111426, -2.3043696280399235),
 'g': (-1.4952976788127375, -0.7692410136151269),
 'e': (-5.618206981392621, 3.5849419654806614),
 'f': (-1.2936336368387211, 0.45886187792471134)}
#_rtg_ = rt.interactiveGraphLayout(df, {'draw_labels':True, 'pos':pos, 'relationships':[('fm','to')]})
#_rtg_
_my_link_ = rt.link(my_df, draw_labels=True, pos=my_pos, relationships=[('fm','to')], w=384, h=384, bounds_percent=0.2)

In [ ]:
def donutLinkNodeOverlap(df, relationships, pos, k=6, iters=100, w=384, h=384, bounds_percent=0.2):
    # Create a link for transformation information
    _link_ = rt.link(df, relationships=relationships, pos=pos, w=w, h=h, bounds_percent=bounds_percent)
    _link_.renderSVG() # force the rendering and related transformations to occur

    # Flatten the nodes out -- assumes that all nodes (and only nodes) are in the pos dictionary
    _node_ls_  = list(pos.keys())
    _node_to_i_ = {}
    for i in range(len(_node_ls_)): _node_to_i_[_node_ls_[i]] = i
    _node_sxy_ = []
    for _node_ in _node_ls_: 
        sx, sy = _link_.xT(pos[_node_][0]), _link_.yT(pos[_node_][1])
        _node_sxy_.append((sx, sy))
    sx_min, sx_max, sy_min, sy_max = sx, sx, sy, sy
    for _node_ in _node_ls_:
        sx_min, sx_max = min(sx_min, sx), max(sx_max, sx)
        sy_min, sy_max = min(sy_min, sy), max(sy_max, sy)

    # Make random cluster centers
    cluster_centers = {}
    for i in range(k): 
        sx, sy = random.random() * (sx_max - sx_min) + sx_min, random.random() * (sy_max - sy_min) + sy_min
        cluster_centers[i] = (sx, sy)

    # Iterate K-Means
    for _iter_ in range(iters):
        # Assign nodes to their closest center
        center_assignments = {}
        for j in range(len(_node_ls_)):
            _node_ = _node_ls_[j]
            min_dist       = (_node_sxy_[j][0] - cluster_centers[0][0])**2 + (_node_sxy_[j][1] - cluster_centers[0][1])**2
            closest_center = 0
            for i in range(1, k):
                dist = (_node_sxy_[j][0] - cluster_centers[i][0])**2 + (_node_sxy_[j][1] - cluster_centers[i][1])**2
                if dist < min_dist:
                    min_dist       = dist
                    closest_center = i
            if closest_center not in center_assignments: center_assignments[closest_center] = []
            center_assignments[closest_center].append(_node_)
        # If there are any centers without nodes, assign a random node to them
        for i in range(k): 
            if i not in center_assignments: center_assignments[i] = [random.choice(_node_ls_)]
        # Update centers
        for i in range(k): 
            sx, sy = 0, 0
            for _node_ in center_assignments[i]: 
                sx, sy = sx + _node_sxy_[_node_to_i_[_node_]][0], sy + _node_sxy_[_node_to_i_[_node_]][1]
            cluster_centers[i] = (sx/len(center_assignments[i]), sy/len(center_assignments[i]))

    # Track nodes to their cluster centers
    node_to_cluster_center = {}
    for i in center_assignments:
        for _node_ in center_assignments[i]: 
            node_to_cluster_center[_node_] = i

    # Create the Voronoi diagram from the cluster centers
    voronoi_pts = []
    for i in cluster_centers: voronoi_pts.append(cluster_centers[i])
    nborhoods = rt.isedgarVoronoi(voronoi_pts, Box=[(0,_link_.h),(_link_.w,_link_.h),(_link_.w,0),(0,0)])

    # Determine the degree information for each node (in and out degree)
    _fm_, _to_         = relationships[0][0], relationships[0][1]
    _fms_, _tos_       = df[_fm_], df[_to_]
    _fm_ext_, _to_ext_ = [], []
    _fm_ext_.extend(_fms_), _to_ext_.extend(_tos_)
    _fm_ext_.extend(_tos_), _to_ext_.extend(_fms_)
    df_degree = pl.DataFrame({_fm_:_fm_ext_, _to_:_to_ext_})
    df_degree = df_degree.with_columns(pl.col(_to_).replace_strict(node_to_cluster_center).alias('to_cluster'))

    df_cluster_counter = rt.polarsCounter(df_degree, [_fm_,'to_cluster'], _to_, count_by_set=True)

    # Uniform color ordering
    color_order = rt.colorRenderOrder(df_degree, 'to_cluster', _to_, count_by_set=True)

    _svg_ = [f'<svg x="0" y="0" width="{_link_.w}" height="{_link_.h}">']
    _svg_.append('<g opacity="0.5">')
    for _poly_i_ in range(len(nborhoods)):
        center_xy = cluster_centers[_poly_i_]
        _poly_ = nborhoods[_poly_i_]
        d = f'M {_poly_[0][0]} {_poly_[0][1]}  '
        for i in range(1, len(_poly_)): d += f'L {_poly_[i][0]} {_poly_[i][1]} '
        d += 'Z'
        _svg_.append(f'<path d="{d}" fill="{rt.co_mgr.getColor(_poly_i_)}" />')
        # _svg_.append(f'<text x="{center_xy[0]}" y="{center_xy[1]}" stroke="#ff0000" text-anchor="middle">{_poly_i_}</text>')
    _svg_.append('</g>')

    for i in center_assignments:
        _nodes_ = set(center_assignments[i])
        _df_    = df_cluster_counter.filter(pl.col(_fm_).is_in(_nodes_))
        for _node_ in _nodes_: 
            sx, sy = _link_.xT(pos[_node_][0]), _link_.yT(pos[_node_][1])
            _df_node_ = _df_.filter(pl.col(_fm_) == _node_)
            _within_  = _df_node_.filter(pl.col('to_cluster') == i)
            _without_ = _df_node_.filter(pl.col('to_cluster') != i)
            _ratio_intras_ = 0.0
            if len(_within_) > 0: _ratio_intras_ = float(_within_['__count__'][0]) / max(len(_nodes_)-1, 1)
            glyph_svg = rt.concentricGlyph(_without_, sx, sy, 0.0, _ratio_intras_, order=color_order, nbor='to_cluster', count_by='__count__')
            _svg_.append(glyph_svg)
    _svg_.append('</svg>')
    return ''.join(_svg_)

rt.tile([donutLinkNodeOverlap(my_df, [('fm','to')], my_pos), _my_link_])

In [ ]:
df_test = pl.DataFrame({'fm':['n', 'n'], 'to_cluster':[3, 4], '__count__':[1, 2]})
df_test = pl.DataFrame({'fm':['n'], 'to_cluster':[4], '__count__':[10]})
rt.tile([
'<svg x="0" y="0" width="128" height="128">' + 
'<rect width="128" height="128" x="0" y="0" fill="#101010" stroke="#ffffff" />' +
rt.concentricGlyph(df_test, 64, 64, 0.6, 0.5, r_min=32.0, r_max=64.0, nbor='to_cluster', count_by='__count__') +
'</svg>'])

In [ ]:
edges_filename  = '../../data/stanford/facebook/348.edges'
_lu_ = {'fm':[], 'to':[]}
for _edge_ in open(edges_filename, 'rt').read().split('\n'):
    if _edge_ == '': continue
    _split_     = _edge_.split()
    _fm_, _to_  = int(_split_[0]), int(_split_[1])
    _lu_['fm'].append(_fm_), _lu_['to'].append(_to_)
df  = pl.DataFrame(_lu_)
g   = rt.createNetworkXGraph(df, [('fm','to')])
#pos = nx.spring_layout(g)
#pos = rt.hyperTreeLayout(g)
pos = nx.kamada_kawai_layout(g)
params = {'df':df, 'relationships':[('fm','to')], 'pos':pos, 'w':512, 'h':512}
rt.tile([donutLinkNodeOverlap(**params), rt.link(**params)])